In [1]:
import cv2
import os
import time
import csv

def record_videos(actions, output_dir):
    cap = cv2.VideoCapture(1)
    
    # Get native resolution of the camera
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    print(f"Resolution: {frame_width}x{frame_height}, FPS: {fps}")

    os.makedirs(output_dir, exist_ok=True)
    
    recording = False
    video_writer = None
    action_idx = 0
    sequence_dir = None
    timestamps = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        if recording:
            video_writer.write(frame)
            current_time = time.time()
            timestamps.append(current_time)
            draw_text_with_background(frame, f'Recording: {actions[action_idx]}', (10, 30), font_scale=0.8,
                                      font_thickness=1, text_color=(255, 255, 255), bg_color=(4, 133, 25))
        else:
            draw_text_with_background(frame, f'Stopped recording: {actions[action_idx]}', (10, 30), font_scale=0.8,
                                      font_thickness=1, text_color=(255, 255, 255), bg_color=(140, 7, 7))

        cv2.imshow('Video Recording', frame)

        key = cv2.waitKey(10) & 0xFF
        if key == ord('r') and not recording:
            sequence_dir = os.path.join(output_dir, actions[action_idx], f"sequence_{get_next_sequence_number(os.path.join(output_dir, actions[action_idx]))}")
            os.makedirs(sequence_dir, exist_ok=True)
            video_path = os.path.join(sequence_dir, "sequence_video.avi")
            video_writer = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
            timestamps = []  # Reset timestamps for new recording
            recording = True
            frame_num = 0
        elif key == ord('s') and recording:
            video_writer.release()
            recording = False
            # Save timestamps to a CSV file
            timestamp_path = os.path.join(sequence_dir, "timestamps.csv")
            with open(timestamp_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Frame", "Timestamp"])
                for i, timestamp in enumerate(timestamps):
                    writer.writerow([i, timestamp])
        elif key == ord('a'):
            action_idx = (action_idx + 1) % len(actions)
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def get_next_sequence_number(action_path):
    try:
        existing_sequences = [int(d.split('_')[1]) for d in os.listdir(action_path) if os.path.isdir(os.path.join(action_path, d)) and d.startswith('sequence_')]
        if existing_sequences:
            return max(existing_sequences) + 1
        else:
            return 0
    except FileNotFoundError:
        return 0

def draw_text_with_background(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, font_thickness=2, text_color=(255, 255, 255), bg_color=(66, 135, 245)):
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    rect_x1 = position[0] - 5
    rect_y1 = position[1] - text_h - 5
    rect_x2 = position[0] + text_w + 5
    rect_y2 = position[1] + 5
    cv2.rectangle(image, (rect_x1, rect_y1), (rect_x2, rect_y2), bg_color, -1)
    cv2.putText(image, text, position, font, font_scale, text_color, font_thickness)

In [3]:
# Example usage
actions = ['normal_shopping', 'shoplifting']
output_dir = 'Recorded_Videos'
record_videos(actions, output_dir)

Resolution: 640x480, FPS: 30
